#  Using Reinforcement Learning to Play Pong

This tutorial demonstrates using reinforcement learning to train an agent to play Pong.  This task isn't directly related to chemistry, but video games make an excellent demonstration of reinforcement learning techniques.

![title](assets/pong.png)

## Colab

This tutorial and the rest in this sequence can be done in Google Colab (although the visualization at the end doesn't work correctly on Colab, so you might prefer to run this tutorial locally). If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Using_Reinforcement_Learning_to_Play_Pong.ipynb)



In [1]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!



Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\users\hp\deepchem_2\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


'2.8.1.dev'

In [2]:
!pip install "gym[atari,accept-rom-license]"

## Reinforcement Learning

Reinforcement learning involves an *agent* that interacts with an *environment*.  In this case, the environment is the video game and the agent is the player.  By trial and error, the agent learns a *policy* that it follows to perform some task (winning the game).  As it plays, it receives *rewards* that give it feedback on how well it is doing.  In this case, it receives a positive reward every time it scores a point and a negative reward every time the other player scores a point.

The first step is to create an `Environment` that implements this task.  Fortunately,
OpenAI Gym already provides an implementation of Pong (and many other tasks appropriate
for reinforcement learning).  DeepChem's `GymEnvironment` class provides an easy way to
use environments from OpenAI Gym.  We could just use it directly, but in this case we
subclass it and preprocess the screen image a little bit to make learning easier.

In [3]:
import deepchem as dc
import numpy as np


class PongEnv(dc.rl.GymEnvironment):
  def __init__(self):
    super(PongEnv, self).__init__('Pong-v4')
    self._state_shape = (80, 80)

  @property
  def state(self):
    # Crop everything outside the play area, reduce the image size,
    # and convert it to black and white.
    state_array = self._state
    cropped = state_array[34:194, :, :]
    reduced = cropped[0:-1:2, 0:-1:2]
    grayscale = np.sum(reduced, axis=2)
    bw = np.zeros(grayscale.shape, dtype=np.float32)
    bw[grayscale != 233] = 1
    return bw

  def __deepcopy__(self, memo):
    return PongEnv()

env = PongEnv()

Next we create a model to implement our policy.  This model receives the current state of the environment (the pixels being displayed on the screen at this moment) as its input.  Given that input, it decides what action to perform.  In Pong there are three possible actions at any moment: move the paddle up, move it down, or leave it where it is.  The policy model produces a probability distribution over these actions.  It also produces a *value* output, which is interpreted as an estimate of how good the current state is.  This turns out to be important for efficient learning.

The model begins with two convolutional layers to process the image.  That is followed by a dense (fully connected) layer to provide plenty of capacity for game logic.  We also add a small Gated Recurrent Unit (GRU).  That gives the network a little bit of memory, so it can keep track of which way the ball is moving.  Just from the screen image, you cannot tell whether the ball is moving to the left or to the right, so having memory is important.

We concatenate the dense and GRU outputs together, and use them as inputs to two final layers that serve as the
network's outputs.  One computes the action probabilities, and the other computes an estimate of the
state value function.

We also provide an input for the initial state of the GRU, and return its final state at the end.  This is required by the learning algorithm.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PongPolicy(dc.rl.Policy):
    def __init__(self):
        super(PongPolicy, self).__init__(['action_prob', 'value', 'rnn_state'], [np.zeros(16, dtype=np.float32)])

    def create_model(self, **kwargs):
        class TestModel(nn.Module):
            def __init__(self):
                super(TestModel, self).__init__()
                # Convolutional layers
                self.conv1 = nn.Conv2d(1, 16, kernel_size=8, stride=4)
                self.conv2 = nn.Conv2d(16, 32, kernel_size=4, stride=2)
                self.fc1 = nn.Linear(2048, 256)
                self.gru = nn.GRU(256, 16, batch_first = True)
                self.action_prob = nn.Linear(272, env.n_actions)
                self.value = nn.Linear(272, 1)
            def forward(self, inputs):
                state = (torch.from_numpy((inputs[0])))
                rnn_state = (torch.from_numpy(inputs[1]))
                reshaped = state.view(-1, 1, 80, 80)
                conv1 = F.relu(self.conv1(reshaped))
                conv2 = F.relu(self.conv2(conv1))
                conv2 = conv2.view(conv2.size(0), -1)
                x = F.relu(self.fc1(conv2))
                reshaped_x = x.view(1, -1, 256)
                #x = torch.flatten(x, 1)
                gru_out, rnn_final_state = self.gru(reshaped_x, rnn_state.unsqueeze(0))
                rnn_final_state = rnn_final_state.view(-1,16)
                gru_out = gru_out.view(-1, 16)
                concat = torch.cat((x, gru_out), dim=1)
                #concat = concat.view(-1, 272)
                action_prob = F.softmax(self.action_prob(concat), dim=-1)
                value = self.value(concat)
                return action_prob, value, rnn_final_state
        return TestModel()
policy = PongPolicy()

We will optimize the policy using the Advantage Actor Critic (A2C) algorithm.  There are lots of hyperparameters we could specify at this point, but the default values for most of them work well on this problem.  The only one we need to customize is the learning rate.

In [5]:
import torch.nn.functional as F
from deepchem.rl.torch_rl.torch_a2c import A2C

from deepchem.models.optimizers import Adam
a2c = A2C(env, policy, model_dir='model', optimizer=Adam(learning_rate=0.0002))

Optimize for as long as you have patience to.  By 1 million steps you should see clear signs of learning.  Around 3 million steps it should start to occasionally beat the game's built in AI.  By 7 million steps it should be winning almost every time.  Running on my laptop, training takes about 20 minutes for every million steps.

In [6]:
# Change this to train as many steps as you have patience for.
a2c.fit(1000)

c:\Users\HP\anaconda3\envs\deep\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Let's watch it play and see how it does! 

In [7]:
# This code doesn't work well on Colab
env.reset()
while not env.terminated:
    env.env.render()
    env.step(a2c.select_action(env.state))

c:\Users\HP\anaconda3\envs\deep\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!